In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score, RocCurveDisplay
import shap
import joblib
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

In [ ]:
# Set general font size
plt.rcParams['font.size'] = '13'

# dpi
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

## version 1) 

daily version: keeping latest datapoint on a day for each pool, inference will run hourly

### dataprep

In [ ]:
df = pd.read_csv('../data/pools.csv')
dfEnriched = pd.read_json('../data/dataEnriched.json')

In [ ]:
df.shape, dfEnriched.shape

In [ ]:
# merging poolinfo columns to df
poolInfoCols = [
    'chain',
    'project',
    'pool',
    'stablecoin', 
    'ilRisk',
    'exposure',
]

df = df.merge(dfEnriched[poolInfoCols], how='left', on=['pool', 'chain', 'project'])

In [ ]:
df.shape

In [ ]:
# step1) remove rows with extreme apy values
df = df[df.apy <= df.apy.quantile(.999)]

In [ ]:
df.shape

In [ ]:
# step2) keep only the latest value on a given day per pool
# reason: i assume the daily fluctuation is stable so less interesting. also this will reduce the dataset by
# a lot but will speed up prototyping. could come back to an hourly version but for now want to keep it simple
df['timestamp'] = pd.to_datetime(df['timestamp'])
# sort before group
df = df.sort_values(['pool', 'timestamp']).reset_index(drop=True)
df = df.groupby(['pool', pd.Grouper(key='timestamp', freq='1D')]).last().reset_index()

In [ ]:
df_full = df.copy()

In [ ]:
df.shape

In [ ]:
# for the target decided to use median instead of avg cause its robust against any remaining outliers
# eg imagine a pool which starts at 20% and declined to 2% within 4weeks, but has a single outlier on a particular 
# day where apy might be 200% -> using an average for that would be crap cause the target will be skewed by that
# single outlier. hence median

In [ ]:
# we want to predict if a pool can keep its apy for the next 30days
horizon = 30
df = df.assign(
    apyMedianShifted=df.groupby('pool')['apy'].apply(lambda x: x.rolling(horizon).median().shift(-(horizon-1)))
)

In [ ]:
df.isnull().sum()

In [ ]:
# given the series is not very large for each pool, lots of missing values on the target (note, this will get
# much better within the month but for now we have a rather small remaining dataset, still sufficient though)
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df[df['apy'] > 1000]['apy'].describe()

In [ ]:
# leaving this, random forest robust against outliers and target is calculated on pool basis anyways

calculate the ML target variable

```
y = {
    0: if ((apyFuture - apy) / apy) < -0.2
    1: else
}
```

so if the pct-change btw forward looking 30day median apy to todays apy is < -20%, we assign label 0 (== pool apy unstable), in all other cases, we consider it stable.
threshold of -20% is just something i thought I'd still consider to be kinda stable

In [ ]:
df['pct'] = (df['apyMedianShifted'] - df['apy']) / df['apy']
df['target'] = np.where(df['pct'] < -0.2, 0, 1)

In [ ]:
# target distribution
df['target'].value_counts().plot.bar()
plt.grid()

### ML

In [ ]:
# add some basic backward looking stats as features:
grouping = df.groupby('pool')
df['apyMeanExpanding'] = grouping['apy'].apply(lambda x: x.expanding().mean())
df['apyStdExpanding'] = grouping['apy'].apply(lambda x: x.expanding().std())

In [ ]:
df.isnull().sum()

In [ ]:
# filling missing std's with 0
df = df.fillna(0)

In [ ]:
# factorize cat variables
cols_to_factorize = ['chain', 'project', 'ilRisk', 'exposure', 'stablecoin']

for i in cols_to_factorize:
    df[f'{i}_factorized'] = pd.factorize(df[i])[0]

In [ ]:
# leaving out all weak features for now
features = [
    'apy',
    'tvlUsd',
    'apyMeanExpanding',
    'apyStdExpanding',
    'chain_factorized',
    'project_factorized',
#     'ilRisk_factorized',
#     'exposure_factorized',
#     'stablecoin_factorized',
]

In [ ]:
random_state = 1993

Pale Rider#9068 provided some useful feedback to make setup more robust and less biassed during performance evaluation. Instead of simply assuming iid (which is a very strong assumption given the data clearly has some
temporal structure I will move away from just randomly splitting into train and test towards
slicing off a test set based on timestamp so that the training data doesn't include any future data)

Also, I think the cross validation process can further be improved by using time series split instead of vanilla
cv. The below is a minimal effort implementation.

In [ ]:
# first sort on timestamp anscending
df = df.sort_values('timestamp').reset_index(drop=True)

In [ ]:
df.timestamp.min(), df.timestamp.max()

In [ ]:
# find an optimal cutoff date so that test set consists of roughly 30% of dataset
g = df.groupby(pd.Grouper(key='timestamp', freq='D'))

In [ ]:
# find split point ('taking last 30% as test set')
g.size().cumsum() / g.size().sum()

In [ ]:
# -> ~2022-03-03 last training day

In [ ]:
cutoff_date = '2022-03-03'

In [ ]:
X_train = df[df.timestamp <= cutoff_date]
X_test = df[df.timestamp > cutoff_date]

y_train = X_train['target']
y_test = X_test['target']

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train.timestamp.min(), X_train.timestamp.max()

In [ ]:
X_test.timestamp.min(), X_test.timestamp.max()

In [ ]:
X_train[features].head()

In [ ]:
y_train.head()

In [ ]:
# using 2 algos with out of box settings, logreg cause basic and random forest cause usually better
# and checking for increase in cv -> increase in test? (as a consistency
# check if test set is similarly distributed to train)

clf_lr = LogisticRegression()
clf_rf = RandomForestClassifier(random_state=random_state, n_estimators=100, n_jobs=-1, oob_score=True)

In [ ]:
random_state = 1993

In [ ]:
ts = TimeSeriesSplit(n_splits=5)

In [ ]:
# sorting data again with second column being pool, so that the sort is deterministic and we get same results
# on repeated runs (sort on timestamp alone might still result in different sort order as not unique)
X_train = X_train.sort_values(["timestamp", 'pool'], ascending=True).reset_index(drop=True)
y_train = X_train['target']

In [ ]:
clf_rf

In [ ]:
features

In [ ]:
d = defaultdict(list)

# running time series split
for algo, algoname in zip([clf_lr, clf_rf], ['logreg', 'randomforest']):
    print(f'Running ts split for {algo}')
    for i, (train_idx, val_idx) in enumerate(ts.split(X_train.values)):

        print(f"train on fold 0-{i}, validate on fold {i+1}")
        print(f"train_idx: {train_idx}, test_idx: {val_idx}")

        X_train_ts, X_val_ts = X_train.loc[train_idx], X_train.loc[val_idx]
        y_train_ts, y_val_ts = X_train_ts['target'], X_val_ts['target']

        algo.fit(X_train_ts[features].values, y_train_ts.values)
        y_pred_val = algo.predict_proba(X_val_ts[features].values)[:, 1]
        roc_score = roc_auc_score(y_val_ts.values, y_pred_val)
        print(f"roc-auc {roc_score:.5f}\n")
        d[algoname].append(roc_score)
        
    print("-" * 100)

In [ ]:
np.mean(d['logreg'])

In [ ]:
np.mean(d['randomforest'])

In [ ]:
# check what features drive performance via feature importance plot
clf_rf.fit(X_train[features].values, y_train.values)

In [ ]:
# check oob score
clf_rf.oob_score_

In [ ]:
importances = clf_rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf_rf.estimators_], axis=0)
forest_importances = pd.Series(importances, index=features).sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(10, 5))
forest_importances.plot.barh(yerr=std, ax=ax)
ax.set_title("feature importances")
ax.set_ylabel("mean decrease in impurity")
plt.grid(True)

In [ ]:
# test performance
y_pred_test = clf_rf.predict_proba(X_test[features].values)[:, 1]
roc_auc_score(y_test, y_pred_test)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
RocCurveDisplay.from_estimator(clf_rf, X_test[features].values, y_test, ax=ax)
plt.grid()

#### error analysis

In [ ]:
# check performance on a project level (want to see if there is a project for which predictions are
# significantly worse than others

In [ ]:
X_train['y_pred'] = clf_rf.predict_proba(X_train[features].values)[:, 1]
X_train['abs_delta'] = abs(X_train['y_pred'] - X_train['target'])

X_test['y_pred'] = clf_rf.predict_proba(X_test[features].values)[:, 1]
X_test['abs_delta'] = abs(X_test['y_pred'] - X_test['target'])

In [ ]:
# on project level
X_train.groupby('project')['abs_delta'].describe().sort_values('75%')

In [ ]:
# on chain level
X_train.groupby('chain')['abs_delta'].describe().sort_values('75%')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8), ncols=2, nrows=2)
X_train['y_pred'].plot.hist(ax=ax[0, 0], title='y-pred distribution [train]')
X_train['abs_delta'].plot.hist(ax=ax[0, 1], title='abs-delta distribution [train]')


X_test['y_pred'].plot.hist(ax=ax[1, 0], title='y-pred distribution [test]')
X_test['abs_delta'].plot.hist(ax=ax[1, 1], title='abs-delta distribution [test]')

ax[0, 0].grid(True)
ax[0, 1].grid(True)
ax[1, 0].grid(True)
ax[1, 1].grid(True)

In [ ]:
# check individual examples of predictions

In [ ]:
def plot_sample(pool: str):
    fig, ax = plt.subplots(figsize=(15, 5))
    df_full[df_full['pool'] == pool].set_index('timestamp')['apy'].plot(ax=ax)
    plt.grid()

In [ ]:
cols = [
    'pool', 
    'timestamp', 
    'chain', 
    'project', 
    'apy', 
    'apyMedianShifted', 
    'pct', 
    'target', 
    'y_pred', 
    'abs_delta',
]
X_train[X_train['abs_delta'] == 0][cols].tail()

In [ ]:
plot_sample('0x53a901d48795C58f485cBB38df08FA96a24669D5')

### shap interpretability

In [ ]:
# shap library is quite slow on large number of trees in the ensemble, i train a second instance of rf
# on a lower nb of estimators to speed up the shapley part
clf_rf_shap = RandomForestClassifier(random_state=random_state, n_estimators=10)
clf_rf_shap.fit(X_train[features].values, y_train.values)

In [ ]:
explainer = shap.TreeExplainer(clf_rf_shap)
shap_values = explainer.shap_values(X_train[features].values, check_additivity=False)

In [ ]:
shap.summary_plot(shap_values[1], X_train[features], plot_type='dot', plot_size=(10, 5))

### interpretation (numerical features only):


+ apy: large values of apy (red) lead to a decrease in probability of a pool to maintain the apy over the next 4weeks. this makes sense as larger apy values are often much more volatile

+ apyMeanExpanding: larger values of apyMeanExpanding (red) increase the probability of a pool to maintain the apy over the next 4weeks.
    again makes sense because if the backward looking mean apy is large then the apy is likely going to be similarly high and stable for the next 4weeks.
    
+ apyStdExpanding: less clear but i'd interpret it this way: the larger the feature value (red) the lower the probability of a pool to maintain the apy over the next 4weeks. more volatility in the past -> less likely of stable apy in the future

+ tvlUsd: bit harder to tell but would say: larger values -> higher probability in prediction which i think is driven by the larger tvl pools with lower apys, but which are more stable overall

In [ ]:
### some sanity checks how consistent the predictions will be, eg are they fluctuating a lot?

In [ ]:
X_test.head()

In [ ]:
df_full[df_full['project'] == 'curve'].sort_values('tvlUsd', ascending=False).iloc[0, :].pool

In [ ]:
p = '0xDC24316b9AE028F1497c275EB9192a3Ea0f67022-ethereum'
a = X_train[X_train['pool'] == p].sort_values('timestamp')[['timestamp', 'target', 'y_pred']]
b = X_test[X_test['pool'] == p].sort_values('timestamp')[['timestamp', 'target', 'y_pred']]

In [ ]:
plot_sample(p)

In [ ]:
pd.concat([a, b]).sort_values('timestamp')

In [ ]:
# depending on pool, less or more fluctuation btw predictions, eg could be that today prediction is very high for
# stable within the next 4weeks, and tomorrow the reverse (if apy is very high tmr for example)
# this will get better as we get more training data. assume within the next 4-6weeks i'd retrain everything with 
# better backward looking statistics

In [ ]:
### explainer dashboard has some cool additional insights:
# checking on test set

In [ ]:
X_test.timestamp.min(), X_test.timestamp.max()

In [ ]:
explainer = ClassifierExplainer(clf_rf_shap, X_test[features], y_test)

In [ ]:
db = ExplainerDashboard(explainer, shap_interaction=False, mode='inline')

In [ ]:
db.run(mode='inline')

### saving model and categorical feature mapping

In [ ]:
# retrain on full historical data
clf_rf.fit(df[features].values, df['target'].values)

In [ ]:
clf_rf.oob_score_

In [ ]:
# save full prepared training data (just as reference)
df[features + ['target']].to_csv("df.csv", index=False)

In [ ]:
# save model
joblib.dump(clf_rf, '../artefacts/clf_1.joblib', compress=3)

In [ ]:
# save feature list
joblib.dump(features, '../artefacts/feature_list.joblib')

In [ ]:
# feature mappings (required in triggerEnrichment)

In [ ]:
project = df[['project', 'project_factorized']].set_index('project')
chain = df[['chain', 'chain_factorized']]

mapping_project = df.set_index('project')[['project_factorized']].to_dict()
mapping_chain = df.set_index('chain')[['chain_factorized']].to_dict()

d = {}

d.update(mapping_project)
d.update(mapping_chain)